In [ ]:
# pip install pandas
# pip install folium

In [1]:
import pandas as pd
import folium
from IPython.display import display
import json

def menu():
    print('<<상권 분석 프로그램>> by 3조\n')
    print('1. 희망하는 업종 서울시 내 분포 시각화')
    print('2. 희망하는 업종, 자본을 토대로 지역 추천받기')

    key = int(input('원하는 내용을 선택하세요(1,2 중 택1)\n'))
    


    if key == 1:
        visualize()
    elif key == 2:
        recommend()
    else:
        print('\n\n\n')
        print('잘못된 접근입니다')
        print('\n\n\n')
        menu()

def visualize(): # 시각화 두 가지 방법 중 선택
    print('1. 서울시 내 업종별 점포 다 찍어서 보기')
    print('2. 서울시 행정동 내 업종 점포 분포 보기')
    choice = int(input('원하시는 항목의 번호를 입력하세요'))
    if choice == 1 :
        plot_upjong()
    elif choice == 2:
        plot_compare()
    else:
        print('\n\n')
        print('잘못 입력하셨습니다. 프로그램을 다시 실행합니다')
        print('\n')
        menu()
    
    
def plot_upjong(): # 점포들을 다 찍어서 보여주는 것
    upjong = int(input('희망하는 업종을 입력하세요\n(1. pc방, 2. 카페, 3. 편의점, 4. 요식업)\n'))

    df = pd.read_csv('위도경도.csv')

    upjong_dict = {
        1 : 'PC방',
        2 : '카페',
        3 : '편의점'
    }

    if upjong in upjong_dict.keys():
        upjong_name = upjong_dict[upjong] # 번호를 이름으로 치환
        data = df[df['업종'] == upjong_name]
        data.index = range(len(data))

        print('\n\n')  
        print('--------------------------------')
        print('--------------------------------')
        print('--------------------------------')
        print('-------지도를 로딩중입니다-------')
        print('--------------------------------')
        print('--------------------------------')
        print('--------------------------------')

        print('\n\n')
        print('서울시 내의 {0}의 개수는 {1}개입니다.\n(출처:소상공인시장진흥공단)'.format(upjong_name, len(data)))
        print('\n\n')
        
        center = [37.541, 126.986]
        
        mm = folium.Map(location=center, zoom_start=12,tiles='cartodbpositron') # 베이스가 되는 맵 저장


        for i in range(len(data)): # 데이터마다 위도,경도에 기반하여 맵에 찍어줌
            folium.CircleMarker(
                location = df.loc[i, ['위도', '경도']],

                radius = 3,
                fill = True,
                fill_color = '#00897b',
                fill_opacity = 0.5,
                color = None
            ).add_to(mm)
            
    
    elif upjong == 4:
        food_cat = int(input('희망하는 요식업 번호를 입력하세요\n(1. 한식, 2. 일식, 3. 양식, 4. 분식, 5. 치킨)\n'))

        food_dict = {
            1 : '한식',
            2 : '일식',
            3 : '양식',
            4 : '분식',
            5 : '치킨'
        }

        food_cat_name = food_dict[food_cat]
        data = df[df['업종'] == food_cat_name]
        data.index = range(len(data))
        
        print('\n\n')  
        print('--------------------------------')
        print('--------------------------------')
        print('--------------------------------')
        print('-------지도를 로딩중입니다-------')
        print('--------------------------------')
        print('--------------------------------')
        print('--------------------------------')

        print('\n\n')
        print('서울시 내의 {0}의 개수는 {1}개입니다.\n(출처:소상공인시장진흥공단)'.format(food_cat_name, len(data)))
        print('\n\n')
        center = [37.541, 126.986]
        mm = folium.Map(location=center, zoom_start=12,tiles='cartodbpositron')


        for i in range(len(data)):
            folium.CircleMarker(
                location = df.loc[i, ['위도', '경도']],

                radius = 3,
                fill = True,
                fill_color = '#00897b',
                fill_opacity = 0.5,
                color = None
            ).add_to(mm)
    
    display(mm)
            
                
def plot_compare():
    
    f = open('geodata.txt', 'r') # json 형태의 str 타입 geo 데이터 불러오기
    data = f.read()
    data = data.replace("\'", "\"") # '로는 에러가 나서 ""로 바꿔줌
    geo = json.loads(data) # str에서 json형식으로

    dong = []
    for data in geo['features']:
        dong.append(data['properties']['adm_nm'].split()[2]) # 데이터와 매칭하는데 필요한 key값인 행정동명을 만들어줌
    
    
    upjong = int(input('희망하는 업종을 입력하세요\n(1. pc방, 2. 카페, 3. 편의점, 4. 요식업)\n'))
                
    upjong_dict = {
        1 : 'PC방',
        2 : '카페',
        3 : '편의점'
    }
    
    if upjong in upjong_dict.keys():

        data = pd.read_csv('위도경도.csv')

        data = data[data['업종'] == upjong_dict[upjong]] # 해당 업종만 가져옴
        data = data.groupby(['행정동명'])['상호명'].count() # count함수로 해당 행정동의 점포 수를 구해줌
        data = pd.merge(pd.DataFrame(dong, columns = ['행정동명']), data, how = 'left', left_on = '행정동명', right_on = '행정동명')
        data.rename(columns = {'상호명' : '점포수'}, inplace = True)
        data = data.drop_duplicates(subset = ['행정동명'])

        seoul_map = folium.Map(location = [37.5502,126.982], zoom_start = 10.5 , tiles = 'cartodbpositron')
        seoul_map.choropleth(geo_data = geo,
                            data = data,
                            columns = ['행정동명','점포수'],
                            fill_color = 'BuPu',
                            key_on = 'properties.sggnm',
                            fill_opacity = 0.5, line_opactiy = 1,
                            legend = '서울시 행정동별 해당 업종 분포')
    

    elif upjong == 4:
        food_cat = int(input('희망하는 요식업 번호를 입력하세요\n(1. 한식, 2. 일식, 3. 양식, 4. 분식, 5. 치킨)\n'))

        food_dict = {
            1 : '한식',
            2 : '일식',
            3 : '양식',
            4 : '분식',
            5 : '치킨'
        }
    
        data = pd.read_csv('위도경도.csv')

        data = data[data['업종'] == food_dict[food_cat]] # 해당 업종만 가져옴
        data = data.groupby(['행정동명'])['상호명'].count() # count함수로 해당 행정동의 점포 수를 구해줌
        data = pd.merge(pd.DataFrame(dong, columns = ['행정동명']), data, how = 'left', left_on = '행정동명', right_on = '행정동명')
        data.rename(columns = {'상호명' : '점포수'}, inplace = True)
        data = data.drop_duplicates(subset = ['행정동명'])

        seoul_map = folium.Map(location = [37.5502,126.982], zoom_start = 10.5 , tiles = 'cartodbpositron')
        seoul_map.choropleth(geo_data = geo,
                            data = data,
                            columns = ['행정동명','점포수'],
                            fill_color = 'BuPu',
                            key_on = 'properties.sggnm',
                            fill_opacity = 0.5, line_opactiy = 1,
                            legend = '서울시 행정동별 해당 업종 분포')
   
    return display(seoul_map)

 
def recommend():
    upjong = int(input('희망하는 업종을 입력하세요\n(1. pc방, 2. 카페, 3. 편의점, 4. 요식업)\n'))
    
    
    if (upjong == 1) | (upjong == 2) | (upjong == 3):
        upjong_recommend(upjong)
        
        
    elif upjong == 4:
        upjong = str(input('희망하는 요식업의 자음을 입력하세요\n(ㄱ. 한식, ㄴ. 일식, ㄷ. 양식, ㄹ. 분식, ㅁ. 치킨)\n'))
        upjong_recommend(upjong)
    
    
        
# def analysis():
#     upjong = int(input('희망하는 업종을 입력하세요\n(1. pc방, 2. 카페, 3. 편의점, 4. 요식업\n)'))
#     jabon = int(input('희망하는 평당 임대료를 입력하세요\n(ex: 150000)'))
#     location = input('희망하는 행정동을 입력하세요\n(ex: 흑석동)')
    
#     data = df[(df['업종'] == 'pc방')  & (df['행정동_코드_명'] == '흑석동')]
    
    
def upjong_recommend(upjong):
    jabon = float(input('희망하는 평당 임대료를 입력하세요\n(ex: 150000)'))
    
    upjong_dict = {
            1 : 'pc방',
            2 : '카페',
            3 : '편의점',
            'ㄱ' : '한식',
            'ㄴ' : '일식',
            'ㄷ' : '양식',
            'ㄹ' : '분식',
            'ㅁ' : '치킨'
        }
        
    df = pd.read_csv('스케일링.csv')
    df = df[df['업종'] == upjong_dict[upjong]]
    df = df[df['임대료'] <= jabon]

    print('\n')

    want_list = ['유동인구', '초등학교', '중학교', '고등학교', '대학교', '지하철역', '버스 정거장', '상주인구']
    want_rank = []



    for i in range(len(want_list)):
        while len(want_list)>0:
            k = 0
            for j in want_list:
                print(str(k+1) + " : " + j)
                k+=1
            want_num = int(input('창업에 있어 가장 우선시하는 요인의 번호를 입력하세요.'))
            if (want_num <= len(want_list)) & (want_num > 0):
                want_rank.append(want_list.pop(want_num-1))
                print('\n')
            else:
                print('\n')
                print('잘못 입력하였습니다')
                print('\n')

    print('당신의 우선순위는 {0} 입니다. 해당 우선순위으로 추천하는 5개 행정동은'.format(want_rank))



    # for want in want_rank:
    #     print('{0}의 등수는 {1}이고 가중치는 {2:.2f}다'.format(want, want_rank.index(want) + 1, (9-want_rank.index(want))/36))
    #     print('\n')

    x1 = (9-want_rank.index('유동인구'))/36
    x2 = (9-want_rank.index('초등학교'))/36
    x3 = (9-want_rank.index('중학교'))/36
    x4 = (9-want_rank.index('고등학교'))/36
    x5 = (9-want_rank.index('대학교'))/36
    x6 = (9-want_rank.index('지하철역'))/36
    x7 = (9-want_rank.index('버스 정거장'))/36
    x8 = (9-want_rank.index('상주인구'))/36


    df['점수'] = ''
    df['점수'] = df['유동인구']*x1 + df['초등학교']*x2 + df['중학교']*x3 + df['고등학교']*x4 + \
    df['대학교']*x5 + df['지하철역']*x6 + df['버스 정거장']*x7 + df['상주인구']*x8 + df['당월_매출_건수'] * 0.3+ df['당월_매출_금액'] * 0.3
    top5 = df.sort_values(by = '점수', ascending = False)['행정동_코드_명'][:5]
    print('\n-------------------------------------------')
    i  = 0
    for place in top5:
        print('{0}번째 추천 구역은 {1} 입니다'.format(i+1, place))
        i+=1
    print('\n-------------------------------------------')
    choice = input('원하는 지역이 추천 지역에 해당되지 않은 이유는?(Y/N)')
    
    if choice == 'Y':
        whynoloc(upjong, jabon, want_rank)
    elif choice == 'N':
        print('프로그램을 종료합니다.')
        
        
    else:
        print('잘못된 명령입니다. 프로그램을 종료합니다')
        
        
        
def whynoloc(upjong, jabon, want_rank):
    
    upjong_dict = {
            1 : 'pc방',
            2 : '카페',
            3 : '편의점',
            'ㄱ' : '한식',
            'ㄴ' : '일식',
            'ㄷ' : '양식',
            'ㄹ' : '분식',
            'ㅁ' : '치킨'
        }
    
    data = pd.read_csv('스케일링.csv')
    data = data[data['업종'] == upjong_dict[upjong]]
    
    loc = input('원하시는 지역의 행정동명을 입력하세요\n(ex: 흑석동)')
    
    if data[data['행정동'] == loc]['임대료'].values:
        
        if (data[data['행정동'] == loc]['임대료'].values > jabon):
            
            print('\n')
            print('{0}의 임대료는 {1}원으로 설정하신 임대료 조건 {2}원보다 높아 제외되었습니다'.format(loc, data[(data['행정동'] == loc)]['임대료'].values, jabon))
            data = data.sort_values(by = want_rank[0], ascending = False)
            print('우선 고려사항 1순위인 {0}의 순위는 424개 행정동 중 {1}위입니다'.format(want_rank[0], list(data['행정동']).index(loc)))
            data = data.sort_values(by = want_rank[1], ascending = False)
            print('우선 고려사항 2순위인 {0}의 순위는 424개 행정동 중 {1}위입니다'.format(want_rank[1], list(data['행정동']).index(loc)))
            data = data.sort_values(by = want_rank[2], ascending = False)
            print('우선 고려사항 3순위인 {0}의 순위는 424개 행정동 중 {1}위입니다'.format(want_rank[2], list(data['행정동']).index(loc)))

        else:
            
            print('\n')
            print('{0}의 임대료는 {1}원으로 설정하신 임대료 조건 {2}원에 해당됩니다.'.format(loc, data[(data['행정동'] == loc)]['임대료'].values, jabon))
            print('우선 고려사항 1순위인 {0}의 순위는 424개 행정동 중 {1}위입니다'.format(want_rank[0], list(data['행정동']).index(loc)))
            data = data.sort_values(by = want_rank[1], ascending = False)
            print('우선 고려사항 2순위인 {0}의 순위는 424개 행정동 중 {1}위입니다'.format(want_rank[1], list(data['행정동']).index(loc)))
            data = data.sort_values(by = want_rank[2], ascending = False)
            print('우선 고려사항 3순위인 {0}의 순위는 424개 행정동 중 {1}위입니다'.format(want_rank[2], list(data['행정동']).index(loc)))
    else:
        print('해당되는 지역의 데이터가 부족합니다. 프로그램을 종료합니다.')
        
menu()

<<상권 분석 프로그램>> by 3조

1. 희망하는 업종 서울시 내 분포 시각화
2. 희망하는 업종, 자본을 토대로 지역 추천받기


원하는 내용을 선택하세요(1,2 중 택1)
 1


1. 서울시 내 업종별 점포 다 찍어서 보기
2. 서울시 행정동 내 업종 점포 분포 보기


원하시는 항목의 번호를 입력하세요 2
희망하는 업종을 입력하세요
(1. pc방, 2. 카페, 3. 편의점, 4. 요식업)
 3


C:\Users\msi\anaconda3\lib\site-packages\folium\folium.py:411: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [2]:
menu()

<<상권 분석 프로그램>> by 3조

1. 희망하는 업종 서울시 내 분포 시각화
2. 희망하는 업종, 자본을 토대로 지역 추천받기


원하는 내용을 선택하세요(1,2 중 택1)
 2
희망하는 업종을 입력하세요
(1. pc방, 2. 카페, 3. 편의점, 4. 요식업)
 2
희망하는 평당 임대료를 입력하세요
(ex: 150000) 150000




1 : 유동인구
2 : 초등학교
3 : 중학교
4 : 고등학교
5 : 대학교
6 : 지하철역
7 : 버스 정거장
8 : 상주인구


창업에 있어 가장 우선시하는 요인의 번호를 입력하세요. 3




1 : 유동인구
2 : 초등학교
3 : 고등학교
4 : 대학교
5 : 지하철역
6 : 버스 정거장
7 : 상주인구


...



1 : 상주인구


창업에 있어 가장 우선시하는 요인의 번호를 입력하세요. 1




당신의 우선순위는 ['중학교', '초등학교', '유동인구', '고등학교', '대학교', '지하철역', '버스 정거장', '상주인구'] 입니다. 해당 우선순위으로 추천하는 5개 행정동은

-------------------------------------------
1번째 추천 구역은 길음2동 입니다
2번째 추천 구역은 소공동 입니다
3번째 추천 구역은 역삼1동 입니다
4번째 추천 구역은 서교동 입니다
5번째 추천 구역은 오륜동 입니다

-------------------------------------------


원하는 지역이 추천 지역에 해당되지 않은 이유는?(Y/N) Y
원하시는 지역의 행정동명을 입력하세요
(ex: 흑석동) 흑석동




흑석동의 임대료는 [99796.]원으로 설정하신 임대료 조건 150000.0원에 해당됩니다.
우선 고려사항 1순위인 중학교의 순위는 424개 행정동 중 91위입니다
우선 고려사항 2순위인 초등학교의 순위는 424개 행정동 중 272위입니다
우선 고려사항 3순위인 유동인구의 순위는 424개 행정동 중 16위입니다
